# Time Series Exploratory Data Analysis (EDA) Template

Reusable notebook for investigating time series datasets:
- Trends, seasonality, decomposition
- Categorical effects (month, weekday, holiday)
- Statistical significance & effect sizes
- Diagnostics (stationarity, autocorrelation, normality)
- Nice visualizations for presentation